# Environment setup

In [9]:
from typing import Type

import torch
from torch import nn
from torch.utils.data import DataLoader

from dataset.dataset_info import ClassifierDatasetInfo, DatasetInfo
from dataset.dataset_type import DatasetType
from dataset.STARCOP_dataset import STARCOPDataset

from files_handler import ModelFilesHandler
from measures import MeasureToolFactory
from measures import ModelType

import os
import sys
os.chdir(r"D:\Projects\studia\polsl_ssi_1\MethaneDetection\Thesis")

## Setup datasets
STARCOPDataset is custom class that derives torch.utils.data.Dataset class. It's defined in dataset module.

In [10]:
def setup_dataloaders(
        data_path: str = r"data",
        batch_size: int = 32,
        train_type = DatasetType.EASY_TRAIN,
        image_info_class: Type[DatasetInfo] = ClassifierDatasetInfo,
        crop_size: int = 1
):
    train_dataset = STARCOPDataset(
        data_path=data_path,
        data_type=train_type,
        image_info_class=image_info_class,
        crop_size=crop_size
    )
    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    test_dataset = STARCOPDataset(
        data_path=data_path,
        data_type=DatasetType.TEST,
        image_info_class=image_info_class,
        crop_size=crop_size
    )
    test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

    return train_dataloader, test_dataloader

## Setup models

### Model class

In [11]:
class MethaNetClassifier(nn.Module):
    def __init__(self, in_channels:int = 9, num_classes:int = 2):
        super(MethaNetClassifier, self).__init__()
        self.pre_conv = nn.Sequential(
            nn.Conv2d(in_channels//1, in_channels//2, kernel_size=1),
            nn.Conv2d(in_channels//2, in_channels//4, kernel_size=1),
            nn.Conv2d(in_channels//4, in_channels//8, kernel_size=1),
        )

        self.conv_layers = nn.Sequential(
            nn.Conv2d(1, 6, kernel_size=2, stride=1, padding=0),  # Input channels = 8
            nn.ReLU(),

            nn.Conv2d(6, 12, kernel_size=3, stride=1, padding=0),
            nn.ReLU(),

            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(12, 16, kernel_size=4, stride=1, padding=0),
            nn.ReLU(),

            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Dropout(p=0.2),

            nn.Conv2d(16, 16, kernel_size=4, stride=1, padding=0),
            nn.ReLU(),

            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.fc_layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(16 * 61 * 61, 64),  # Adjust dimensions for 512x512 input
            nn.ReLU(),

            nn.Linear(64, 32),
            nn.ReLU(),

            nn.Linear(32, num_classes),
            nn.Softmax(dim=1)  # Softmax for class probabilities
        )

    def forward(self, x):
        x = self.pre_conv(x)
        x = self.conv_layers(x)
        x = self.fc_layers(x)
        return x


### Prepare models

In [12]:
def setup_model(model: nn.Module, lr: float, device: str):
    model = model.to(device)

    criterion = nn.CrossEntropyLoss()  # Binary classification
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    return model, criterion, optimizer

## Prepare training function

In [13]:
def print_progress_bar(percentage, loss):
    bar_length = 50  # Length of the progress bar
    filled_length = int(bar_length * percentage // 100)
    bar = '=' * filled_length + '-' * (bar_length - filled_length)
    sys.stdout.write(f"\r[{bar}] {percentage:.2f}% [Loss: {loss:.6f}]")
    sys.stdout.flush()

In [14]:
def train(criterion, device, epochs, model, optimizer, dataloader, model_handler, log_batches: bool = False):
    model.train()
    for epoch in range(epochs):  
        print(f"Epoch: {epoch}")
        running_loss = 0.0
        for batch_id, (images, mag1c, labels) in enumerate(dataloader): 
            optimizer.zero_grad()

            input_image = torch.cat((images, mag1c), dim=1).to(device)
            labels = labels.long().to(device)

            outputs = model(input_image)

            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            
            if log_batches and (batch_id + 1) % 10 == 0:
                print_progress_bar(batch_id / len(dataloader) * 100, running_loss / (batch_id + 1))

        print(f"Epoch {epoch + 1}, Loss: {running_loss / len(dataloader)}")
        model_handler.save_raw_model(model)

## Prepare evaluate function

In [15]:
def evaluate(criterion, device, model, dataloader, measurer):
    model.eval()
    all_predictions = []
    all_labels = []
    running_loss = 0.0

    for batch_id, (images, mag1c, labels) in enumerate(dataloader):
        input_image = torch.cat((images, mag1c), dim=1).to(device)
        labels = labels.long().to(device)

        outputs = model(input_image)
        predictions = torch.argmax(outputs, dim=1)
        loss = criterion(outputs, labels)

        running_loss += loss.item()
        all_predictions.append(predictions.cpu().detach())
        all_labels.append(labels.cpu().detach())

    measures = measurer.compute_measures(torch.cat(all_predictions), torch.cat(all_labels))
    print(f"Validation loss: {running_loss / len(dataloader)}.\nMeasures:\n{measures}")
    return measures

# Train model

In [16]:
epochs = 15
device = "cuda" if torch.cuda.is_available() else "cpu"
lr = 1e-4

# Override for file handler
from models.Convolutional.MethaNet.model import MethaNetClassifier

train_dataloader, test_dataloader = setup_dataloaders(train_type=DatasetType.TRAIN)
model = MethaNetClassifier()
model, criterion, optimizer = setup_model(model, lr, device)
model_handler = ModelFilesHandler()
measurer = MeasureToolFactory.get_measure_tool(ModelType.CNN)

train(criterion, device, epochs, model, optimizer, train_dataloader, model_handler, log_batches=True)
measures = evaluate(criterion, device, model, test_dataloader, measurer)

Epoch: 0
[=============================================-----] 91.67% [Loss: 0.529073]Epoch 1, Loss: 0.520691869435487
Epoch: 1
[=============================================-----] 91.67% [Loss: 0.388527]Epoch 2, Loss: 0.38788151023564515
Epoch: 2
[=============================================-----] 91.67% [Loss: 0.352972]Epoch 3, Loss: 0.35187796889631834
Epoch: 3
[=============================================-----] 91.67% [Loss: 0.333890]Epoch 4, Loss: 0.33372077456227056
Epoch: 4
[=============================================-----] 91.67% [Loss: 0.329118]Epoch 5, Loss: 0.3286008478866683
Epoch: 5
[=============================================-----] 91.67% [Loss: 0.329275]Epoch 6, Loss: 0.3284068637424045
Epoch: 6
[=============================================-----] 91.67% [Loss: 0.328966]Epoch 7, Loss: 0.32845190581348205
Epoch: 7
[=============================================-----] 91.67% [Loss: 0.325241]Epoch 8, Loss: 0.3246684118553444
Epoch: 8
[===================================

# Save model

In [17]:
model_handler.save_model(
    model=model,
    metrics = measures,
    model_type=ModelType.CNN,
    epoch=epochs,
)

'trained_models\\model_cnn_2024_12_14_13_20_07.pickle'

In [19]:
easy_train_dataloader, _ = setup_dataloaders(train_type=DatasetType.EASY_TRAIN)

measures = evaluate(criterion, device, model, train_dataloader, measurer)


Validation loss: 0.3235416580681448.
Measures:
         TP        FP        FN        TN  Precision  Sensitivity  \
0  0.493431  0.004964  0.003504  0.498102   0.990041     0.992949   

   Specificity       NPV       FPR        Accuracy   F-Score             IoU  \
0     0.990133  0.993015  0.009867  tensor(0.9915)  0.991493  tensor(0.9831)   

       MCC       AUC          CI  
0  0.98307  0.991541  (1.0, 1.0)  
